In [1]:
import numpy as np

candles = np.load("data/bar_deap_v1.npy")
candles.shape

(5789, 6)

In [2]:
from src.features.simple_feature_calculator import SimpleFeatureCalculator

feature_calculator = SimpleFeatureCalculator()
feature_calculator.load(candles, sequential=False)
feature_calculator_seq = SimpleFeatureCalculator()
feature_calculator_seq.load(candles, sequential=True)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/jesse/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [3]:
from src.features.simple_feature_calculator.buildin.feature_names import BUILDIN_FEATURES

dt_feats = [f"{i}_dt" for i in BUILDIN_FEATURES]
ddt_feats = [f"{i}_ddt" for i in BUILDIN_FEATURES]
lag_feats = []
for l in range(1, 21):
    lags = [f"{f}_lag{l}" for f in BUILDIN_FEATURES]
    lag_feats.extend(lags)
dt_lag_feats = []
for l in range(1, 21):
    lags = [f"{f}_lag{l}" for f in dt_feats]
    dt_lag_feats.extend(lags)
ddt_lag_feats = []
for l in range(1, 21):
    lags = [f"{f}_lag{l}" for f in ddt_feats]
    ddt_lag_feats.extend(lags)

all_feats = BUILDIN_FEATURES + dt_feats + ddt_feats + lag_feats + dt_lag_feats + ddt_lag_feats

features = {}

for f in all_feats:
    res = feature_calculator.get([f])
    features = {**features, **res}

In [4]:
import pandas as pd

df = pd.DataFrame(features)
df

,adx_7,adx_14,aroon_diff,ac_0,ac_1,ac_2,ac_3,ac_4,ac_5,ac_6,...,vmd_w64_2_ddt_lag20,vmd_w128_0_ddt_lag20,vmd_w128_1_ddt_lag20,vmd_w128_2_ddt_lag20,vmd_w256_0_ddt_lag20,vmd_w256_1_ddt_lag20,vmd_w256_2_ddt_lag20,vmd_w512_0_ddt_lag20,vmd_w512_1_ddt_lag20,vmd_w512_2_ddt_lag20
0,31.207778,21.241031,-28.571429,0.920769,0.849959,0.783941,0.731531,0.694256,0.669413,0.652755,...,364.455818,-142.897249,128.041009,279.78312,1838.189434,-384.037794,293.546295,-3487.057215,881.057642,176.331007


In [4]:
df.to_parquet("data/features.parquet")

### 确认实际的nan数量

In [1]:
import numpy as np

from src.indicators.prod import VMD_NRBO
from src.indicators.prod.wavelets.cls_cwt_swt import CWT_SWT

merged_bar = np.load("data/merged_bar.npy")
print(merged_bar.shape)

vmd_nrbo = VMD_NRBO(merged_bar, 32, sequential=True)
cwt_swt = CWT_SWT(merged_bar, 32, sequential=True)

vmd_nrbo_single = VMD_NRBO(merged_bar, 32, sequential=False)
cwt_swt_single = CWT_SWT(merged_bar, 32, sequential=False)

(15063, 6)


In [7]:
single_res = cwt_swt_single.res(dt=True, lag=1)
sequential_res = cwt_swt.res(dt=True, lag=1)[-1]

single_res - sequential_res

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.]])

In [3]:
vmd_nrbo_single.res()

array([[-660.2680612 ,  136.15266516,   67.72702177]])

In [ ]:
import numpy as np

from src.indicators.prod import _fill_gap

dt_result = [i[1:] - i[:-1] for i in cwt_swt.raw_result]
dt_result = np.array([i[-1] for i in dt_result])
dt_result = _fill_gap(dt_result, cwt_swt.candles)

In [11]:
dt_result.shape

(15063, 21)

In [1]:
import json

with open("strategies/BinanceBtcEntropyBarV1/feature_info.json", "r") as f:
    feature_info = json.load(f)

all_features = []
all_features.extend(feature_info["meta"]["meta"])
all_features.extend(feature_info["side"]["long"])
all_features.extend(feature_info["side"]["short"])

all_features = sorted(list(set(all_features)))
len(all_features)

343

In [2]:
import pandas as pd

df = pd.read_parquet("data/features.parquet")
df[all_features].isna().sum().sort_values(ascending=False)

ac_1                                0
price_variance_ratio_dt_lag3        0
reactivity_lag3                     0
reactivity_lag2                     0
reactivity_lag1                     0
                                   ..
evenbetter_sinewave_long            0
ehlers_early_onset_trend_lag15      0
ehlers_early_onset_trend_lag1       0
ehlers_early_onset_trend_dt_lag2    0
williams_r_lag6                     0
Length: 343, dtype: int64

### 新特征探索

In [6]:
import numpy as np

merged_bar = np.load("data/merged_bar.npy")
close_prices = merged_bar[:, 2]

In [9]:
from src.indicators.prod.wavelets import cwt

res = cwt(merged_bar, window=32, source_type="close", sequential=False)
np.isnan(res).sum()

0

In [17]:
res.ndim

1